In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

import sklearn.metrics as metrics

from sklearn.metrics import confusion_matrix

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import warnings

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import auc, make_scorer, f1_score

import pickle

#oversampling
warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:
class Test:

    def __init__(self):
        pass

    x_train, x_test, y_train, y_test = None, None, None, None
    model = None
    model_path = None
    loaded_model = None
    classes = ['Below 10%', 'Above 10%']

    def load_dataset(self):
        self.x_train = pd.read_csv("x_train.csv")
        self.x_test = pd.read_csv("x_test.csv")
        self.y_train = pd.read_csv("y_train.csv")
        self.x_test = pd.read_csv("y_test.csv")
        return self

    def train_model(self):
        params = {}
        model = SVC(**params)
        model.fit(self.x_train,self.y_train)
        self.model = model
        print("Model trained successfully")
        return self

    def load_model(self):
        self.loaded_model = pickle.load(open(self.model_path, 'rb'))
        print("model loaded successfully")
        return self
    
    @staticmethod
    def model_metrics(y_true, y_pred, y_probab):
        print("--------------------------------")
        print("\nModel Evaluation:")
        print("accurancy: ",metrics.accuracy_score(y_true,y_pred))
        print("precision: ",metrics.precision_score(y_true,y_pred,average='macro'))
        print("recall: ",metrics.recall_score(y_true,y_pred,average='micro'))
        print("f-measure: ",metrics.f1_score(y_true,y_pred,average='weighted'))
        print("--------------------------------")

    @staticmethod
    def plot_conf_matrix(classes, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.xticks(np.arange(len(classes)) + 0.5, classes)
        plt.yticks(np.arange(len(classes)) + 0.5, classes, rotation=0)
        plt.show()
    
    @staticmethod
    def plot_roc_curve(y_probab, y_true):

        probabilities = y_probab

        # Compute ROC curve and ROC AUC for each class
        n_classes = probabilities.shape[1]
        roc_auc = []
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_true, probabilities[:, i], pos_label=i)
            roc_auc.append(auc(fpr, tpr))

        # Compute average ROC AUC score
        average_roc_auc = sum(roc_auc) / n_classes

        # Print ROC AUC score for each class
        for i, auc_score in enumerate(roc_auc):
            print(f"Class {i} ROC AUC: {auc_score}")

        # Print average ROC AUC score
        print("Average ROC AUC:", average_roc_auc)

        # Plot ROC curve for each class
        plt.figure(figsize=(8, 6))
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_true, probabilities[:, i], pos_label=i)
            plt.plot(fpr, tpr, label=f'Class {i} (ROC AUC = {roc_auc[i]:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', label='Random')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve for Each Class')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

    @staticmethod
    def plot_precission_recal_curve(self):
        pass
    
    def test_model(self,load=False):
            if load:
                model = self.loaded_model
            else:
                model = self.model
            result = cross_val_score(model, self.x_train, self.y_train, cv=6, scoring='accuracy')
            print("Cross validatation Accuracy: ", np.mean(result))
            y_pred = model.predict(self.x_test)
            print(classification_report(self.y_test, y_pred,target_names=self.classes))
            y_pred_probab = model.predict_proba(self.x_test)
            self.model_metrics(self.y_test,y_pred,y_pred_probab[: , 1])
            self.plot_conf_matrix(self.classes,self.y_test,y_pred)
            self.plot_roc_curve(y_pred_probab,self.y_test)
            return self
    
    def loaded_model_pipeline(self):
        self.load_dataset().load_model().test_model(load=True)

    def new_model_pipeline(self):
        self.load_dataset().train_model().test_model()


In [ ]:
Test1 = Test()
Test2 = Test()

In [ ]:
Test1.loaded_model_pipeline()

In [ ]:
Test2.new_model_pipeline()